### Complex Numbers

When measurement models involve complex numbers, care must be taken to ensure maintaining the correlations between real and imaginary components and conversion from real/imaginary to magnitude/phase space.
Suncal provides `suncal.ModelComplex` for models involving complex values. (Note complex models are not available in the GUI at this time.)

With complex models, the `measure` method takes both value and uncertainty as arguments. Only Normal distributions are supported currently, but the correlation between real and imaginary components may be specified. This calculation shows an uncertainty evaluation in real and imaginary components.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import suncal

In [2]:
model = suncal.ModelComplex('f = re(a*b) + a/b')
model.var('a').measure(3+2j, uncertainty=.25+.25j, correlation=.35)
model.var('b').measure(4-3j, uncertainty=.25+.25j, correlation=.35)
result = model.calculate()
result

|Function   | Method       | Nominal        | Standard Uncertainty   | Correlation  |
|----------|------------|--------------|----------------------|-------------|
|f          | GUM          | 18.240+0.680j  | ± 0.033+0.033j         | 0.5491       |
|f          | Monte Carlo  | 18.240+0.680j  | ± 0.033+0.033j         | 0.5487       |

In [3]:
model = suncal.Model('f = 0', 'g = x')
model.var('x').measure(10).typeb(unc=1)
result = model.calculate()
result

|Function   | Method       | Nominal   | Std. Uncertainty   | 95% Coverage   | k         | Deg. Freedom  |
|----------|------------|---------|------------------|--------------|---------|--------------|
|$f$   | GUM          | 0.0  | 0.0           | ± 0.0     | 1.960     | inf           |
|$f$   | Monte Carlo  | 0.0  | 0.0           | (0.0, 0.0) | nan       | &nbsp;        |
|$g$   | GUM          | 10.0  | 1.0           | ± 2.0     | 1.960     | inf           |
|$g$   | Monte Carlo  | 10.0 | 1.00          | (8.0, 12.0) | 1.959     | &nbsp;        |

If the results are desired in magnitude and phase format, provide `magphase=True` to the Model. This must be done when the model is created, so that appropriate functions may be added for converting real and imaginary into magnitude and phase while properly accounting for their uncertainties.

In [4]:
model = suncal.ModelComplex('f = a * b', magphase=True)
model.var('a').measure(3+2j, uncertainty=.25+.25j, correlation=.35)
model.var('b').measure(4-3j, uncertainty=.25+.25j, correlation=.35)
result = model.calculate()
result

|Function   | Method       | Nominal          | Standard Uncertainty   | Correlation  |
|----------|------------|----------------|----------------------|-------------|
|f          | GUM          | 18.03 ∠-3.2 rad  | ± 0.81 ∠2.3 rad        | 0.1227       |
|f          | Monte Carlo  | 18.04 ∠-3.2 rad  | ± 0.81 ∠2.3 rad        | 0.1231       |

If degrees are preferred instead of radians:

In [5]:
result.degrees(True)

|Function   | Method       | Nominal       | Standard Uncertainty   | Correlation  |
|----------|------------|-------------|----------------------|-------------|
|f          | GUM          | 18.03 ∠-3.2°  | ± 0.81 ∠2.3°           | 0.1227       |
|f          | Monte Carlo  | 18.04 ∠-3.2°  | ± 0.81 ∠2.3°           | 0.1231       |

Finally, if the measured values are also given in magnitude and phase, they must be appropriately converted to real and imaginary to handle the statistics. Suncal provides the `measure_magphase` method to define variables:

In [6]:
model = suncal.ModelComplex('f = a + b', magphase=True)
model.var('a').measure_magphase(magnitude=0.95, phase=45, degrees=True, umagnitude=.05, uphase=5)
model.var('b').measure_magphase(magnitude=0.95, phase=45, degrees=True, umagnitude=.05, uphase=5)
result = model.calculate()

Internally, Suncal splits each model function, and each model variable, into a real part and imaginary part, with appropriate correlations applied. The normal GUM and Monte Carlo methods are computed on the split functions, then the results are recombined. Because every function and variable becomes multiple model equations, the calculations can become slow. To see the full calculation, use `result.report.fullreport`.

In [7]:
result.report.fullreport.variables

|Variable   | Mean      | Std. Uncertainty   | Deg. Freedom   | Description  |
|----------|---------|------------------|--------------|-------------|
|$a_{deg}$   | 0.785  | 0.044           | inf       |              |
|$a_{mag}$   | 0.950  | 0.025           | inf       |              |
|$b_{deg}$   | 0.785  | 0.044           | inf       |              |
|$b_{mag}$   | 0.950  | 0.025          | inf      |              |

In [8]:
result.report.fullreport.summary()

Function,Method,Nominal,Std. Uncertainty,95% Coverage,k,Deg. Freedom
$f_{mag}$,GUM,1.900,0.035,± 0.069,1.960,inf
$f_{mag}$,Monte Carlo,1.900,0.035,"(1.831, 1.969)",1.961,
$f_{rad}$,GUM,0.01371,0.00054,± 0.0011,1.960,inf
$f_{rad}$,Monte Carlo,0.01371,0.00054,"(0.01265, 0.01476)",1.962,
